---
title: 'Exploratory Data Analysis'
categories: ['EDA']
---

In [ ]:
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt

train_file_names = os.listdir("train/")
train_file_names.sort()

train_files = []
for file in train_file_names:
    train_files.append(pd.read_csv(f"train/{file}", sep=";"))

test_file_names = os.listdir("test/")
test_file_names.sort()

test_files = []
for file in test_file_names:
    test_files.append(pd.read_csv(f"test/{file}", sep=";"))

test_files[0].head()

## Distribution of Training Data

In [ ]:
import plotnine as p9

(
    p9.ggplot(train_files[0], p9.aes(x="Value1", fill="Value1", color="Value1"))
    + p9.geom_density(color="red", alpha=0.1)
    + p9.labs(title="Distribution of Train File 1", x="Value 1", y="Density")
).show()

The training data is not normally distributed but we suspect its a combination 
of two normal distributions:
- long-term trend
- short-term fluctuations

We can break up the plot into the specific regions by using a **Fast Fourier Transform**


In [ ]:
import plotnine as p9

(
    p9.ggplot(train_files[0].reset_index(), p9.aes(y="Value1", x="index"))
    + p9.geom_line()
    + p9.geom_smooth(method="loess")
).show()

In [ ]:
from scipy.fft import fft, ifft, fftfreq

fft = fft(train_files[0]["Value1"])
n = fftfreq(len(train_files[0]))
t = train_files[0].index
cutoff = 0.05
long_term_fft = fft.copy()
long_term_fft[np.abs(n) > cutoff] = 0

short_term_fft = fft.copy()
short_term_fft[np.abs(n) <= cutoff] = 0

long_term_signal = np.real(ifft(long_term_fft))
short_term_signal = np.real(ifft(short_term_fft))

plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1)
plt.plot(t, train_files[0]["Value1"], color="gray")
plt.title("Original Signal")

plt.subplot(3, 1, 2)
plt.plot(t, long_term_signal, color="green")
plt.title("Long-Term")

plt.subplot(3, 1, 3)
plt.plot(t, short_term_signal, color="orange")
plt.title("Short-Term")

plt.tight_layout()

plt.show()

That didn't work exactly as we wanted.

We will try to reconstruct the signal as
$$
F(x)=asin(b\theta) + dsin(c\alpha) 
$$

In [ ]:
plt.plot(train_files[0]["Value1"])

f = 3 * np.sin(0.02 * np.arange(1, 2000)) + 33

plt.plot(f)
plt.show()

In [ ]:
signal = train_files[0]["Value1"]
Q1 = np.quantile(signal, 0.25)
Q3 = np.quantile(signal, 0.75)
plt.plot(signal)
plt.axhline(Q1)
plt.axhline(Q3)
plt.show()

## Distribution of Testing Data

In [ ]:
signal = test_files[0].reset_index()

(p9.ggplot(signal, p9.aes(x="Value1")) + p9.geom_density()).show()

## Conclusion

The training data cannot be used to predict the testing data for two reasons:

1. The training data is 2000 observations per file totaling 20,000 compared to 
    the testing data which is 10,000 per file totaling 100,000

2. The training data does not follow the same distribution as the testing data
    meaning a statistical model which makes an assumption about the distribution
    isn't appropriate

The biggest problem is that the training data is so much less than the testing
data.

If we combine the files together, the train-test split won't be 80% and 20%
for train and test respectively but will be the other way around i.e:

- train = 17% of all data
- test = 83% of all data

To balance it properly would require sampling strategies but the goal is 
anomaly detection so oversampling to balance anomalies would add a lot of noise
to the data.

The training data also has no anomalies to detect so the algorithm won't learn
from the data itself.